In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from collections import defaultdict
#import re

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import collections


from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from pathlib import Path
import time

sns.set()

**Dataset**


- [6-campaign](./data/13_campaign.npz)
```markdown

ADBench campaign dataset


```

( more : https://www.kaggle.com/datasets/creepycrap/finance-dataset )

In [2]:
data_dir = Path('./data')
work_with = "5_campaign.npz"

data = np.load(data_dir / work_with)

In [3]:
x=data['X']
y=data['y']

df = pd.DataFrame(x)
df['class'] = y

In [4]:
df

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,class
0,0.209877,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.0,0.0,0.0,1.000000,0.882307,0.376569,0.980730,1.000000,0
1,0.296296,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.0,0.0,0.0,1.000000,0.484412,0.615063,0.981183,1.000000,0
2,0.246914,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.0,0.0,0.0,0.937500,0.698753,0.602510,0.957379,0.859735,0
3,0.160494,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.142857,0.0,1.0,0.0,0.333333,0.269680,0.192469,0.150759,0.512287,0
4,0.530864,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.0,0.0,0.0,0.333333,0.340608,0.154812,0.174790,0.512287,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,0.271605,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.0,0.0,0.0,0.333333,0.269680,0.192469,0.158694,0.512287,0
41184,0.333333,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.000000,1.0,0.0,0.0,0.687500,0.389322,0.368201,0.767853,0.877883,1
41185,0.172840,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.000000,1.0,0.0,0.0,0.937500,0.698753,0.602510,0.956926,0.859735,0
41186,0.148148,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,1.0,0.0,0.0,1.000000,0.882307,0.376569,0.980503,1.000000,0


In [5]:
# data distribution 
# 0 - no-fraus, 1 - fraud

print(f'{df["class"].value_counts()}')

class
0    36548
1     4640
Name: count, dtype: int64


```markdown
we can also sub-sample the outliers
```

In [6]:
# sample non_increasing class
target = 'class'
inbound_df = df[df[target] == 0]
outlier_df = df[df[target] == 1].sample(n=int(0.014*len(inbound_df)), random_state=42)

df = pd.concat([inbound_df, outlier_df])
print(f'Sampled: {df[target].value_counts()}')

Sampled: class
0    36548
1      511
Name: count, dtype: int64


In [7]:
# handle categorical data
cat_columns = df.select_dtypes(include=['object'], exclude=["number"]).columns
print(f"uniques val count:\n{df[[*cat_columns]].nunique()}")

uniques val count:
Series([], dtype: float64)


In [8]:
# encode student 
# df = pd.get_dummies(df, columns=["student"], prefix=["ST"],dtype="int8")

In [9]:
cat_columns = df.select_dtypes(include=['object'], exclude=["number"]).columns

# # encode Unnamed: 0 and nameDest with label encoder
from sklearn.preprocessing import LabelEncoder

for col in cat_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    
df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,class
0,0.209877,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0000,0.882307,0.376569,0.980730,1.000000,0
1,0.296296,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.0000,0.484412,0.615063,0.981183,1.000000,0
2,0.246914,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.9375,0.698753,0.602510,0.957379,0.859735,0


In [10]:
target = "class"

X = df.drop([target], axis=1)
y = df[target]

X.shape, y.shape

((37059, 62), (37059,))

In [11]:
# scaling the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X[X.columns] = scaler.fit_transform(X)

X.shape, y.shape

((37059, 62), (37059,))

In [12]:
# split the data into train and test
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y,
#                                                     stratify=y,
#                                                     test_size=0.2,
#                                                     random_state=42)

X_train, X_test, y_train, y_test = X, X, y, y

# count fraud in train and test 
print(f"Train: \n {y_train.value_counts()}")
print(f"Test: \n {y_test.value_counts()} ")

Train: 
 class
0    36548
1      511
Name: count, dtype: int64
Test: 
 class
0    36548
1      511
Name: count, dtype: int64 


In [13]:
''' Helper functions '''

def get_scores(y_true, y_pred):
    scores = {
        'accuracy': round(accuracy_score(y_true, y_pred),2),
        'balanced': round(balanced_accuracy_score(y_true, y_pred),2),
        'F1': round(f1_score(y_true, y_pred),2),
        'precision': round(precision_score(y_true, y_pred),2),
        'recall': round(recall_score(y_true, y_pred),2),
        'roc_auc': round(roc_auc_score(y_true, y_pred),2),
        'pr_auc': round(average_precision_score(y_true, y_pred),2)
    }
    
    return scores

In [14]:
combined_scores = []

isolation forest

In [15]:
# isolation forest

from pyod.models.iforest import IForest

start_time = time.time()

clf_name = 'IForest'

clf = IForest()
clf.fit(X_train)

duration = round(time.time() - start_time,2)

# get the prediction labels and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

# get the prediction on the test data
y_test_pred = clf.predict(X_test)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(X_test)  # outlier scores

scores = {"clf_name":clf_name, **get_scores(y_test, y_test_pred), "duration": duration}
combined_scores.append(scores)

Local Outlier Factor

In [16]:
#  Local Outlier Factor

from pyod.models.lof import LOF

start_time = time.time()

clf_name = 'LOF'
clf = LOF()
clf.fit(X_train)
duration = round(time.time() - start_time,2)

# get the prediction labels
y_train_pred = clf.labels_  
y_test_pred = clf.predict(X_test)


scores = {"clf_name":clf_name, **get_scores(y_test, y_test_pred), "duration": duration}
combined_scores.append(scores)
# scores

ECOD

In [17]:
#  ECOD

from pyod.models.ecod import ECOD
start_time = time.time()


clf_name = 'ECOD'
clf = ECOD()
clf.fit(X_train)
duration = round(time.time() - start_time,2)

# get the prediction labels
y_train_pred = clf.labels_  
y_test_pred = clf.predict(X_test)


scores = {"clf_name":clf_name, **get_scores(y_test, y_test_pred), "duration": duration}
combined_scores.append(scores)

Local Correlation Integral

In [18]:
# #  Local Correlation Integral (LOCI)

# from pyod.models.loci import LOCI
# start_time = time.time()


# clf_name = 'LOCI'
# clf = LOCI()
# clf.fit(X_train)
# duration = round(time.time() - start_time,2)

# # get the prediction labels
# y_train_pred = clf.labels_  
# y_test_pred = clf.predict(X_test)


# scores = {"clf_name":clf_name, **get_scores(y_test, y_test_pred), "duration": duration}
# combined_scores.append(scores)

LSCP: Locally Selective Combination of Parallel Outlier Ensembles

In [19]:
# #  LSCP

# from pyod.models.lscp import LSCP
# from pyod.models.lof import LOF
# start_time = time.time()


# clf_name = 'LSCP'
# detector_list = [LOF(), LOF()]
# clf = LSCP(detector_list)
# clf.fit(X_train)
# duration = round(time.time() - start_time,2)

# # get the prediction labels
# y_train_pred = clf.labels_  
# y_test_pred = clf.predict(X_test)


# scores = {"clf_name":clf_name, **get_scores(y_test, y_test_pred), "duration": duration}
# combined_scores.append(scores)

COPOD: Copula-Based Outlier Detection

In [20]:
#  COPOD: Copula-Based Outlier Detection

from pyod.models.copod import COPOD
start_time = time.time()


clf_name = 'COPOD'
clf = COPOD()
clf.fit(X_train)
duration = round(time.time() - start_time,2)

# get the prediction labels
y_train_pred = clf.labels_  
y_test_pred = clf.predict(X_test)


scores = {"clf_name":clf_name, **get_scores(y_test, y_test_pred), "duration": duration}
combined_scores.append(scores)

ABOD: Angle-Based Outlier Detection

In [21]:
#  ABOD: Angle-Based Outlier Detection

from pyod.models.abod import ABOD
start_time = time.time()

clf_name = 'ABOD'
clf = ABOD()
clf.fit(X_train)
duration = round(time.time() - start_time,2)

# get the prediction labels
y_train_pred = clf.labels_  
y_test_pred = clf.predict(X_test)


scores = {"clf_name":clf_name, **get_scores(y_test, y_test_pred), "duration": duration}
combined_scores.append(scores)

QMCD: Quasi-Monte Carlo Discrepancy outlier detection

In [22]:
# QMCD: Quasi-Monte Carlo Discrepancy outlier detection

from pyod.models.qmcd import QMCD
start_time = time.time()

clf_name = 'QMCD'
clf = QMCD()
clf.fit(X_train)
duration = round(time.time() - start_time,2)

# get the prediction labels
y_train_pred = clf.labels_  
y_test_pred = clf.predict(X_test)


scores = {"clf_name":clf_name, **get_scores(y_test, y_test_pred), "duration": duration}
combined_scores.append(scores)


Rapid distance-based outlier detection via sampling

In [23]:
# MAD - Rapid distance-based outlier detection via sampling

from pyod.models.sampling import Sampling
start_time = time.time()

clf_name = 'Rapid distance-based'
clf = Sampling()
clf.fit(X_train)
duration = round(time.time() - start_time,2)

# get the prediction labels
y_train_pred = clf.labels_  
y_test_pred = clf.predict(X_test)


scores = {"clf_name":clf_name, **get_scores(y_test, y_test_pred), "duration": duration}
combined_scores.append(scores)



## Book Keeping

In [24]:
# save the results

save_dir = Path('results')
save_dir.mkdir(exist_ok=True)
save_as = save_dir / f"6-{Path(work_with).stem}.csv"

scores_df = pd.DataFrame(combined_scores)
scores_df.to_csv(save_as, index=False)


In [25]:
display(scores_df)

,clf_name,accuracy,balanced,F1,precision,recall,roc_auc,pr_auc,duration
0,IForest,0.90,0.63,0.09,0.05,0.36,0.63,0.03,1.91
1,LOF,0.90,0.56,0.05,0.03,0.20,0.56,0.02,4.05
2,ECOD,0.90,0.69,0.12,0.07,0.48,0.69,0.04,2.26
3,COPOD,0.90,0.69,0.12,0.07,0.48,0.69,0.04,1.25
4,ABOD,0.88,0.64,0.08,0.05,0.40,0.64,0.03,15.32
5,QMCD,0.90,0.66,0.10,0.06,0.42,0.66,0.03,9.60
6,Rapid distance-based,0.90,0.67,0.11,0.06,0.44,0.67,0.03,0.04


``` markdown

# methods used
1. [Isolation Forest](https://cs.nju.edu.cn/zhouzh/zhouzh.files/publication/icdm08b.pdf)
2. ...
```